# The Double Pendulum

In [1]:
(require '[clojupyter.misc.helper :as helper])
(run! helper/add-dependencies '[[org.clojure/data.json "0.2.6"]
                                [net.littleredcomputer/sicmutils "0.11.0-SNAPSHOT"]
                                [thi.ng/geom "0.0.908"]])
(ns double-pendulum
    (:refer-clojure :exclude [partial zero? + - * / ref])
    (:require [sicmutils.env :refer :all]
              [clojupyter.misc.display :as display]
              [thi.ng.geom.viz.core :as viz]
              [thi.ng.geom.svg.core :as svg]))

In [2]:
(defn V
  [m_1 m_2 l_1 l_2 g]
  (fn [[_ [theta phi] _]]
    (let [y_1 (- (* l_1 (cos theta)))
          y_2 (- y_1 (* l_2 (cos phi)))]
      (+ (* m_1 g y_1)
         (* m_2 g y_2)))))

#'double-pendulum/V

In [3]:
(defn T
  [m_1 m_2 l_1 l_2 _]
  (fn [[_ [theta phi] [thetadot phidot]]]
    (let [v1sq (* (square l_1) (square thetadot))
          v2sq (* (square l_2) (square phidot))]
      (+ (* 1/2 m_1 v1sq)
         (* 1/2 m_2 (+ v1sq
                      v2sq
                      (* 2 l_1 l_2 thetadot phidot (cos (- theta phi)))))))))

#'double-pendulum/T

In [4]:
(def L (- T V))
(tex$ ((L 'm_1 'm_2 'l_1 'l_2 'g) 
       (up 't 
           (up 'theta 'phi) 
           (up 'thetadot 'phidot))))

In [5]:
(defn state-derivative [m_1 m_2 l_1 l_2 g]
    (Lagrangian->state-derivative
        (L m_1 m_2 l_1 l_2 g)))

#'double-pendulum/state-derivative

In [6]:
(def equations-of-motion
    ((state-derivative 'm_1 'm_2 'l_1 'l_2 'g)
               (up 't
                   (up 'theta 'phi)
                   (up 'thetadot 'phidot))))

#'double-pendulum/equations-of-motion

In [7]:
(tex$$ (nth equations-of-motion 2))

In [8]:
(def out (atom []))
(defn observe [t [_ [theta phi]]] (swap! out conj [t theta phi]))
(tex$$ ((evolve state-derivative 1 1 1 1 9.8)
        (up 0 (up (/ pi 2) 0) (up 0 0))
        observe
        1/60
        10
        1e-6
        :compile true))

In [9]:
(def spec
  {:x-axis (viz/linear-axis
            {:domain [(first (first @out)) (first (last @out))]
             :range  [50 (- 985 10)]
             :major 1
             :pos    550})
   :y-axis (viz/linear-axis
            {:domain      [(- Math/PI) Math/PI]
             :range       [550 20]
             :major       1
             :minor       0.2
             :pos         50
             :label-dist  15
             :label-style {:text-anchor "end"}})
   :grid   {:attribs {:stroke "#caa"}
            :minor-x true
            :minor-y false}
   :data   [{:values  (map #(vector (nth % 0) (nth % 1)) @out)
             :attribs {:stroke "#0af" :stroke-width "2pt"}
             :layout  viz/svg-line-plot}
            {:values  (map #(vector (nth % 0) (nth % 2)) @out)
             :attribs {:stroke "#f60" :stroke-width "2pt"}
             :layout  viz/svg-line-plot}]})

#'double-pendulum/spec

In [10]:
(defn export-viz
  [spec]
  (->> spec
       (viz/svg-plot2d-cartesian)
       (svg/svg {:width 985 :height 600})
       (svg/serialize)))

(-> spec
    export-viz
    display/make-html)

<?xml version="1.0"?>
 <polyline fill="none" points="50.00,152.50 51.54,152.61 53.08,152.96 54.63,153.53 56.17,154.34 57.71,155.37 59.25,156.63 60.79,158.13 62.33,159.85 63.88,161.81 65.42,164.00 66.96,166.42 68.50,169.08 70.04,171.99 71.58,175.14 73.13,178.54 74.67,182.21 76.21,186.16 77.75,190.39 79.29,194.94 80.83,199.83 82.38,205.09 83.92,210.75 85.46,216.86 87.00,223.43 88.54,230.47 90.08,237.91 91.63,245.60 93.17,253.32 94.71,260.89 96.25,268.17 97.79,275.14 99.33,281.80 100.88,288.17 102.42,294.30 103.96,300.19 105.50,305.89 107.04,311.39 108.58,316.71 110.13,321.85 111.67,326.80 113.21,331.56 114.75,336.11 116.29,340.43 117.83,344.51 119.38,348.32 120.92,351.84 122.46,355.06 124.00,357.95 125.54,360.49 127.08,362.66 128.63,364.44 130.17,365.81 131.71,366.75 133.25,367.25 134.79,367.28 136.33,366.82 137.88,365.85 139.42,364.34 140.96,362.27 142.50,359.63 144.04,356.43 145.58,352.76 147.13,348.76 148.67,344.65 150.21,340.64 151.75,336.87 153.29,333.40 154.83,330.22 156.38,327.31 157.92,324.64 159.46,322.18 161.00,319.88 162.54,317.73 164.08,315.70 165.63,313.75 167.17,311.87 168.71,310.05 170.25,308.25 171.79,306.48 173.33,304.70 174.88,302.91 176.42,301.09 177.96,299.24 179.50,297.34 181.04,295.39 182.58,293.37 184.13,291.27 185.67,289.10 187.21,286.85 188.75,284.51 190.29,282.09 191.83,279.57 193.38,276.97 194.92,274.29 196.46,271.54 198.00,268.71 199.54,265.82 201.08,262.87 202.63,259.88 204.17,256.87 205.71,253.84 207.25,250.80 208.79,247.78 210.33,244.79 211.88,241.84 213.42,238.96 214.96,236.16 216.50,233.45 218.04,230.87 219.58,228.42 221.13,226.12 222.67,223.99 224.21,222.05 225.75,220.31 227.29,218.80 228.83,217.52 230.38,216.49 231.92,215.72 233.46,215.22 235.00,215.01 236.54,215.10 238.08,215.49 239.63,216.19 241.17,217.22 242.71,218.59 244.25,220.31 245.79,222.39 247.33,224.87 248.88,227.78 250.42,231.15 251.96,235.03 253.50,239.50 255.04,244.63 256.58,250.52 258.13,257.21 259.67,264.57 261.21,272.19 262.75,279.54 264.29,286.26 265.83,292.26 267.38,297.59 268.92,302.35 270.46,306.63 272.00,310.49 273.54,313.99 275.08,317.17 276.63,320.08 278.17,322.74 279.71,325.17 281.25,327.39 282.79,329.42 284.33,331.26 285.88,332.94 287.42,334.46 288.96,335.82 290.50,337.04 292.04,338.13 293.58,339.08 295.13,339.90 296.67,340.59 298.21,341.17 299.75,341.63 301.29,341.97 302.83,342.20 304.38,342.32 305.92,342.33 307.46,342.23 309.00,342.02 310.54,341.72 312.08,341.31 313.63,340.79 315.17,340.19 316.71,339.48 318.25,338.68 319.79,337.79 321.33,336.81 322.88,335.75 324.42,334.60 325.96,333.39 327.50,332.10 329.04,330.74 330.58,329.33 332.13,327.86 333.67,326.35 335.21,324.79 336.75,323.21 338.29,321.59 339.83,319.96 341.38,318.31 342.92,316.65 344.46,314.99 346.00,313.33 347.54,311.68 349.08,310.03 350.63,308.39 352.17,306.77 353.71,305.14 355.25,303.53 356.79,301.91 358.33,300.28 359.88,298.65 361.42,296.99 362.96,295.31 364.50,293.58 366.04,291.80 367.58,289.96 369.13,288.03 370.67,286.00 372.21,283.86 373.75,281.57 375.29,279.13 376.83,276.49 378.38,273.63 379.92,270.50 381.46,267.07 383.00,263.27 384.54,259.04 386.08,254.33 387.63,249.11 389.17,243.44 390.71,237.54 392.25,231.80 393.79,226.55 395.33,221.97 396.88,218.11 398.42,214.94 399.96,212.39 401.50,210.41 403.04,208.98 404.58,208.05 406.13,207.60 407.67,207.60 409.21,208.04 410.75,208.89 412.29,210.15 413.83,211.79 415.38,213.79 416.92,216.14 418.46,218.80 420.00,221.76 421.54,224.99 423.08,228.45 424.63,232.14 426.17,236.00 427.71,240.02 429.25,244.17 430.79,248.43 432.33,252.77 433.88,257.17 435.42,261.62 436.96,266.10 438.50,270.60 440.04,275.12 441.58,279.65 443.13,284.21 444.67,288.79 446.21,293.41 447.75,298.09 449.29,302.84 450.83,307.68 452.38,312.66 453.92,317.80 455.46,323.15 457.00,328.73 458.54,334.58 460.08,340.70 461.63,347.02 463.17,353.37 464.71,359.55 466.25,365.34 467.79,370.64 469.33,375.41 470.88,379.67 472.42,383.43 473.96,386.75 475.50,389.65 477.04,392.15 478.58,394.28 480.13,396.06 481.67,397.49 483.21,398.58 484.75,399.35 486.29,399.79 487.